In [1]:
import pandas as pd

# school_df = pd.read_csv('../data/social_explorer_raw/School_Digger_2018.csv')
# school_df = school_df.drop(index=0)
# school_df = school_df.reset_index(drop=True)
# school_df = school_df[['FIPS', 'Qualifying Name', 'State', 'LEA', 'SchoolDigger Rank Elementary', 'SchoolDigger Rank High', 'SchoolDigger Rank Middle']]
# school_df = school_df.dropna()
# school_df = school_df.reset_index()
# 
# 
# # school_df['SchoolDigger Rank Elementary'] = school_df['SchoolDigger Rank Elementary'].astype('float').round(2)
# # school_df['SchoolDigger Rank Middle'] = school_df['SchoolDigger Rank Middle'].astype('float').round(2)
# # school_df['SchoolDigger Rank High'] = school_df['SchoolDigger Rank High'].astype('float').round(2)
# # 
# # school_df.to_pickle('../data/social_explorer/school_df.pkl')
# # # TODO school qualifying name is not always a county but instead school districts so it does not match up clean with other data, need to clean before using
# # 
# # distinct_fips = school_df['FIPS'].nunique()
# # print(distinct_fips)
# # filtered_df = school_df[school_df['Qualifying Name'].str.contains('')]
# filtered_df = school_df.loc[school_df['State'] == "01"]
# school_df.head(100)

In [50]:
crime_df = pd.read_csv('../data/social_explorer_raw/crime.csv')
crime_df = crime_df.drop(index=0)
crime_df = crime_df.dropna()
crime_df = crime_df.reset_index(drop=True)

crime_df['Total Violent and Property Crimes '] = crime_df['Total Violent and Property Crimes '].astype('float')
crime_df['Total Population (2014 est.)'] = crime_df['Total Population (2014 est.)'].astype('float')
crime_df['Total Violent and Property Crimes'+' %'] = (crime_df['Total Violent and Property Crimes '] / crime_df['Total Population (2014 est.)']) * 100


# crime_df = crime_df[['Qualifying Name', 'Total Violent and Property Crimes %']].round(2)
crime_df = crime_df[['FIPS', 'Qualifying Name','Total Violent and Property Crimes %']].round(2)
crime_df.to_pickle('../data/social_explorer/crime_df.pkl')
crime_df.head(100)

,FIPS,Qualifying Name,Total Violent and Property Crimes %
0,01001,"Autauga County, Alabama",3.19
1,01003,"Baldwin County, Alabama",2.22
2,01005,"Barbour County, Alabama",2.82
3,01007,"Bibb County, Alabama",1.11
4,01009,"Blount County, Alabama",2.42
...,...,...,...
95,04011,"Greenlee County, Arizona",2.00
96,04012,"La Paz County, Arizona",2.86
97,04013,"Maricopa County, Arizona",3.44
98,04015,"Mohave County, Arizona",3.32


Demographic data:
We shrink the demographic dataset to focus on population number of each ethnic group of the latest year(year 2020). We do not care about historical data as we are not interested in analyzing trends. Therefore we omit cols such as 2010 total population and ethic population of ethnic group in 2010.

In [39]:
demographic_df = pd.read_csv('../data/social_explorer_raw/Census2020_PL94_Redistricting_Data.csv')
demographic_df = demographic_df.drop(index=0)
demographic_df = demographic_df.reset_index(drop=True)

demographic_col_to_keep = ['FIPS', 'Qualifying Name','2020 Total Population','2020 Population Density', '2020 Housing Units',
       '2020 Occupied Units', '2020 Vacant Units',
       'White 2020', 'African American or Black 2020',
       'American Indian and Alaska Native 2020', 'Asian 2020',
       'Native Hawaiian and Other Pacific Islander 2020', 'Other 2020',
       'Two or More Races 2020',
       'Non-Hispanic 2020', 'Hispanic 2020']
demographic_df_filtered = demographic_df[demographic_col_to_keep]

columns_to_convert = [
       'White 2020', 'African American or Black 2020',
       'American Indian and Alaska Native 2020', 'Asian 2020',
       'Native Hawaiian and Other Pacific Islander 2020',
       'Other 2020', 'Two or More Races 2020',
       'Non-Hispanic 2020', 'Hispanic 2020'
]

demographic_df_filtered = demographic_df_filtered.dropna()
demographic_df_filtered['2020 Total Population'] = demographic_df_filtered['2020 Total Population'].astype('float')

for col in columns_to_convert:
    demographic_df_filtered[col] = demographic_df_filtered[col].astype('float')
    demographic_df_filtered[col+' %'] = (demographic_df_filtered[col] / demographic_df_filtered['2020 Total Population']) * 100

demographic_col_to_keep = ['FIPS','Qualifying Name','2020 Total Population','2020 Population Density',
       'White 2020 %', 'African American or Black 2020 %',
       'American Indian and Alaska Native 2020 %', 'Asian 2020 %',
       'Native Hawaiian and Other Pacific Islander 2020 %', 'Other 2020 %',
       'Two or More Races 2020 %', 'Hispanic 2020 %']
demographic_df = demographic_df_filtered[demographic_col_to_keep].round(2)
demographic_df.to_pickle('../data/social_explorer/demographic_df.pkl')

The latest year of housing data available on Social Explorer was year 2020. This dataset include property rental data such as fair market rent.

In [40]:
housing_df = pd.read_csv('../data/social_explorer_raw/Income_limits_and_fair_market_rent_2020.csv')
housing_df = housing_df.drop(index=0)
housing_df = housing_df.reset_index(drop=True)

housing_col_to_keep = ['FIPS', 'Qualifying Name', 'Median Family Income (5-year ACS)', 'Fair Market Rent for One Bedroom',
       'Fair Market Rent for Two Bedrooms',
       'Fair Market Rent for Three Bedrooms',
       'Fair Market Rent for Four Bedrooms']
housing_df = housing_df[housing_col_to_keep]
housing_df.to_pickle('../data/social_explorer/housing_df.pkl')

In [44]:
environment_df = pd.read_csv('../data/social_explorer_raw/Environmental_summaries_2011.csv')
environment_df = environment_df.drop(index=0)
environment_df = environment_df.reset_index(drop=True)

col_to_keep = ['FIPS', 'Qualifying Name', 'Total Area of Land Coverage', 'Area Of Land Cover Class 11, Open Water'
       , 'Area Of Land Cover Class 12, Perennial Ice/Snow', 'Area Of Land Cover Class 21, Developed, Open Space', 'Area Of Land Cover Class 22, Developed, Low Intensity', 'Area Of Land Cover Class 23, Developed, Medium Intensity', 'Area Of Land Cover Class 24, Developed, High Intensity', 'Area Of Land Cover Class 31, Barren Land (Rock/Sand/Clay)','Area Of Land Cover Class 41, Deciduous Forest','Area Of Land Cover Class 42, Evergreen Forest','Area Of Land Cover Class 43, Mixed Forest', 'Area Of Land Cover Class 52, Shrub/Scrub', 'Area Of Land Cover Class 71, Grassland/Herbaceous', 'Area Of Land Cover Class 81, Pasture/Hay', 'Area Of Land Cover Class 82, Cultivated Crops', 'Area Of Land Cover Class 90, Woody Wetlands', 'Area Of Land Cover Class 95, Emergent Herbaceous Wetlands']

environment_df = environment_df[col_to_keep]

environment_df = environment_df.fillna(0)
environment_df['Total Area of Land Coverage'] = environment_df['Total Area of Land Coverage'].astype('float')

columns_to_convert = ['Area Of Land Cover Class 11, Open Water'
       , 'Area Of Land Cover Class 12, Perennial Ice/Snow', 'Area Of Land Cover Class 21, Developed, Open Space', 'Area Of Land Cover Class 22, Developed, Low Intensity', 'Area Of Land Cover Class 23, Developed, Medium Intensity', 'Area Of Land Cover Class 24, Developed, High Intensity', 'Area Of Land Cover Class 31, Barren Land (Rock/Sand/Clay)','Area Of Land Cover Class 41, Deciduous Forest','Area Of Land Cover Class 42, Evergreen Forest','Area Of Land Cover Class 43, Mixed Forest', 'Area Of Land Cover Class 52, Shrub/Scrub', 'Area Of Land Cover Class 71, Grassland/Herbaceous', 'Area Of Land Cover Class 81, Pasture/Hay', 'Area Of Land Cover Class 82, Cultivated Crops', 'Area Of Land Cover Class 90, Woody Wetlands', 'Area Of Land Cover Class 95, Emergent Herbaceous Wetlands']

for col in columns_to_convert:
    environment_df[col] = environment_df[col].astype('float')
    environment_df[col+' %'] = (environment_df[col] / environment_df['Total Area of Land Coverage']) * 100

col_to_keep = ['FIPS', 'Qualifying Name', 'Total Area of Land Coverage', 'Area Of Land Cover Class 11, Open Water %'
       , 'Area Of Land Cover Class 12, Perennial Ice/Snow %', 'Area Of Land Cover Class 21, Developed, Open Space %', 'Area Of Land Cover Class 22, Developed, Low Intensity %', 'Area Of Land Cover Class 23, Developed, Medium Intensity %', 'Area Of Land Cover Class 24, Developed, High Intensity %', 'Area Of Land Cover Class 31, Barren Land (Rock/Sand/Clay) %','Area Of Land Cover Class 41, Deciduous Forest %','Area Of Land Cover Class 42, Evergreen Forest %','Area Of Land Cover Class 43, Mixed Forest %', 'Area Of Land Cover Class 52, Shrub/Scrub %', 'Area Of Land Cover Class 71, Grassland/Herbaceous %', 'Area Of Land Cover Class 81, Pasture/Hay %', 'Area Of Land Cover Class 82, Cultivated Crops %', 'Area Of Land Cover Class 90, Woody Wetlands %', 'Area Of Land Cover Class 95, Emergent Herbaceous Wetlands %']    

environment_df = environment_df[col_to_keep].round(2)

shortened_columns = {
    'Total Area of Land Coverage': 'Total Area',
    'Area Of Land Cover Class 11, Open Water %': 'Open Water %',
    'Area Of Land Cover Class 12, Perennial Ice/Snow %': 'Ice/Snow %',
    'Area Of Land Cover Class 21, Developed, Open Space %': 'Developed Open Space %',
    'Area Of Land Cover Class 22, Developed, Low Intensity %': 'Developed Low Intensity %',
    'Area Of Land Cover Class 23, Developed, Medium Intensity %': 'Developed Medium Intensity %',
    'Area Of Land Cover Class 24, Developed, High Intensity %': 'Developed High Intensity %',
    'Area Of Land Cover Class 31, Barren Land (Rock/Sand/Clay) %': 'Barren Land %',
    'Area Of Land Cover Class 41, Deciduous Forest %': 'Deciduous Forest %',
    'Area Of Land Cover Class 42, Evergreen Forest %': 'Evergreen Forest %',
    'Area Of Land Cover Class 43, Mixed Forest %': 'Mixed Forest %',
    'Area Of Land Cover Class 52, Shrub/Scrub %': 'Shrub/Scrub %',
    'Area Of Land Cover Class 71, Grassland/Herbaceous %': 'Grassland/Herbaceous %',
    'Area Of Land Cover Class 81, Pasture/Hay %': 'Pasture/Hay %',
    'Area Of Land Cover Class 82, Cultivated Crops %': 'Cultivated Crops %',
    'Area Of Land Cover Class 90, Woody Wetlands %': 'Woody Wetlands %',
    'Area Of Land Cover Class 95, Emergent Herbaceous Wetlands %': 'Emergent Herbaceous Wetlands %'
}

environment_df.rename(columns=shortened_columns, inplace=True)
environment_df = environment_df.fillna(0)
environment_df.to_pickle('../data/social_explorer/environment_df.pkl')

0
